In [23]:
import pandas as pd
import numpy as np
import mne

In [24]:
# Path to your CSV file
csv_file = 'test.csv'

# Read CSV while skipping metadata lines and stripping extra spaces
df = pd.read_csv(csv_file, comment='%', skipinitialspace=True)
df.columns = df.columns.str.strip()

# Optionally, inspect the DataFrame to ensure correct reading
print(df.head())
print(df.dtypes)


   Sample Index  EXG Channel 0  EXG Channel 1  EXG Channel 2  EXG Channel 3  \
0           0.0  110752.849073            0.0            0.0            0.0   
1           1.0  110728.306857            0.0            0.0            0.0   
2           2.0  110659.552891            0.0            0.0            0.0   
3           3.0  110643.683153            0.0            0.0            0.0   
4           4.0  110713.889982            0.0            0.0            0.0   

   EXG Channel 4  EXG Channel 5  EXG Channel 6  EXG Channel 7  \
0            0.0            0.0            0.0            0.0   
1            0.0            0.0            0.0            0.0   
2            0.0            0.0            0.0            0.0   
3            0.0            0.0            0.0            0.0   
4            0.0            0.0            0.0            0.0   

   Accel Channel 0  ...  Other.3  Other.4  Other.5  Other.6  Analog Channel 0  \
0            0.154  ...    255.0    176.0     14.0   

In [25]:
# Define the EXG channel names (assumed to be "EXG Channel 0" ... "EXG Channel 7")
exg_channels = [f'EXG Channel {i}' for i in range(8)]



In [26]:
# Ensure the columns are numeric
for ch in exg_channels:
    df[ch] = pd.to_numeric(df[ch], errors='coerce')

# Convert raw values to Volts:
# - The scaling factor 0.0223 converts raw counts to microvolts.
# - To convert microvolts to volts, multiply by 1e-6.
scaling_factor = 0.0223e-6  
exg_data = df[exg_channels].to_numpy() * scaling_factor


In [27]:
# Set the sampling frequency (from your metadata, here 250 Hz)
sfreq = 250

# Create MNE info structure (using 'eeg' channel type)
info = mne.create_info(ch_names=exg_channels, sfreq=sfreq, ch_types=['eeg'] * len(exg_channels))


In [28]:
info

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [29]:
# Create the MNE Raw object (data shape must be [n_channels, n_times])
raw = mne.io.RawArray(exg_data.T, info)


Creating RawArray with float64 data, n_channels=8, n_times=14703
    Range : 0 ... 14702 =      0.000 ...    58.808 secs
Ready.


In [ ]:
# Rename the channels of the EEG sensors
rename_dict = {
    "EXG Channel 0": "Fz",
    "EXG Channel 1": "Cz",
    "EXG Channel 2": "Pz",
    "EXG Channel 3": "Oz",
    "EXG Channel 4": "T7",
    "EXG Channel 5": "T8",
    "EXG Channel 6": "P7",
    "EXG Channel 7": "P8",
}

raw.rename_channels(rename_dict)

In [31]:
channel_1_data = raw.get_data(picks='Fz')
print(channel_1_data)

ValueError: picks ('Fz') could not be interpreted as channel names (no channel "['Fz']"), channel types (no type "Fz" present), or a generic type (just "all" or "data")

In [ ]:
# Apply a 50 Hz notch filter to remove line noise
raw.notch_filter(freqs=50)

# Plot the raw data (MNE will display amplitudes in µV by default)
raw.plot()


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Using qt as 2D backend.


Channels marked as bad:
none
